In [4]:
# !pip3 install ai.cs
# !pip3 install ai.fri3d

In [6]:
import numpy as np
from ai import cs
from ai.fri3d.model import StaticFRi3D
from astropy import units as u
from matplotlib import pyplot as plt
from PIL import Image
#from mpl_toolkits.mplot3d import proj3d

In [7]:
#%% PLUTO setup

v_unit = 250.0                #km/s
l_unit = u.au.to(u.km, 1)   #in km
t_unit = l_unit/v_unit      #in sec

dt = t_unit*1.88e-4          #aprrox value in sec: has to be greater than the real g_dt
theta_res = 2.0               #in degree
phi_res = 2.0                 #in degree
#v_cme = 1000.0                #velocity of injecting CME in km/s


cme_phi = u.deg.to(u.rad, 9.0+180)
cme_theta = u.deg.to(u.rad, -7.0)  



hw_rad = u.deg.to(u.rad, 43.0)
hh_rad = u.deg.to(u.rad, 43.0)
v_t = 870    #v_cme/(1 + np.tan(hh_rad))

In [8]:
time = dt*np.arange(0, 240, 1)
radius = 0.081 + u.km.to(u.au, v_t*time)
cr = 2284
output_dir = f"Input/CME1"
input_dir = f"Input/CME1/"
output_dir_gif = f"{cr}/Input_gif/CME_trial_periodic_new/"
outputgif = f"{cr}/Input_gif/CME_trial_periodic_new/Input.gif"

In [9]:
for i in range (0, len(radius)):
    sfr = StaticFRi3D(toroidal_height=u.au.to(u.m, radius[i]),
        # poloidal_height=u.au.to(u.m, 0.4),
        latitude=cme_theta,
        longitude=cme_phi,
        half_width=hw_rad,
        half_height=hh_rad,
        pancaking=0.6,
        tilt = u.deg.to(u.rad, 0),
        chirality=-1.0,
        polarity=-1.0,
        flattening=0.5,
        flux=5e12,
        twist=2)
    
    _phi = np.linspace(-u.deg.to(u.rad, 90) + cme_phi, u.deg.to(u.rad, 90) + cme_phi, 181)
    _theta = np.linspace(-u.deg.to(u.rad, 60) + cme_theta, u.deg.to(u.rad, 60) + cme_theta, 121)
    _r = u.au.to(u.m, 0.1)
    
    r, theta, phi = np.meshgrid(_r, _theta, _phi)
    
    x, y, z = cs.sp2cart(r.flatten(), theta.flatten(), phi.flatten())
    
    b, v = sfr.data(x, y, z)
    b *= 1e9
    
    B_mag = np.sqrt(b[:,0]**2 + b[:,1]**2 + b[:,2]**2)
    
    m = np.logical_and(B_mag>0, radius[i]>0.08)
    
    vr = v_t * (v[m,0] + v[m,1]*np.cos(phi.flatten()[m]))

    print("\nSlice number : ", i)
    print('No. of points = "', len(x[m]),'" at', radius[i], 'AU')
    print('Vr = ', np.max(vr))                     #toroidal component
    #print('V_p = ', np.mean(v[m,1])*velocity)                     #poloidal component
    
    theta_pluto = u.deg.to(u.rad, 90) - theta.flatten()[m]
    phi_pluto = u.deg.to(u.rad, 180) + phi.flatten()[m]  
    # phi_pluto = u.deg.to(u.rad) + phi.flatten()[m]
    
#     phi_pluto = np.mod(phi_pluto, 2*np.pi)   # To make sure phi is in the range 0 to 2*pi
    phi_pluto = np.where(phi_pluto == 2*np.pi, 2*np.pi, np.mod(phi_pluto, 2*np.pi))

#     theta_pluto = np.mod(theta_pluto, np.pi) ##############
#     phi_pluto = np.where(phi_pluto > 2*np.pi,phi_pluto - 2*np.pi,phi_pluto)
#     phi_pluto = np.where(phi_pluto < 0,phi_pluto + 2*np.pi,phi_pluto)
    
    sin_t = np.sin(theta_pluto)
    sin_p = np.sin(phi_pluto)
    cos_t = np.cos(theta_pluto)
    cos_p = np.cos(phi_pluto)
    
    Br = b[m,0]*sin_t*cos_p + b[m,1]*sin_t*sin_p + b[m,2]*cos_t
    Bt = b[m,0]*cos_t*cos_p + b[m,1]*cos_t*sin_p - b[m,2]*sin_t
    Bp = -1*b[m,0]*sin_p + b[m,1]*cos_p
    Vr = vr
    
    fg1 = open(f"{output_dir}/FR_Br_"+str(i)+".txt", "w")
    fg2 = open(f"{output_dir}/FR_Bt_"+str(i)+".txt", "w")
    fg3 = open(f"{output_dir}/FR_Bp_"+str(i)+".txt", "w")
    fg4 = open(f"{output_dir}/FR_Vr_"+str(i)+".txt", "w")
    fg5 = open(f"{output_dir}/FR_theta_"+str(i)+".txt", "w")
    fg6 = open(f"{output_dir}/FR_phi_"+str(i)+".txt", "w")
    
    for j in range(0, len(Br)):
        
        fg1.write("%lf\n"%(Br[j]))
        fg2.write("%lf\n"%(Bt[j]))
        fg3.write("%lf\n"%(Bp[j]))
        fg4.write("%lf\n"%(Vr[j]))
        fg5.write("%lf\n"%(theta_pluto[j]))
        fg6.write("%lf\n"%(phi_pluto[j]))
        
    fg1.close()
    fg2.close()
    fg3.close()
    fg4.close()
    fg5.close()
    fg6.close()


Slice number :  0
No. of points = " 5699 " at 0.081 AU
Vr =  740.5897111800452

Slice number :  1
No. of points = " 5893 " at 0.08165424 AU
Vr =  746.0103960077528

Slice number :  2
No. of points = " 6077 " at 0.08230848 AU
Vr =  751.3716759166629

Slice number :  3
No. of points = " 6245 " at 0.08296272 AU
Vr =  756.6496778529261

Slice number :  4
No. of points = " 6447 " at 0.08361696 AU
Vr =  761.843521209532

Slice number :  5
No. of points = " 6623 " at 0.0842712 AU
Vr =  766.9580453454244

Slice number :  6
No. of points = " 6817 " at 0.08492544 AU
Vr =  771.9937129580558

Slice number :  7
No. of points = " 6987 " at 0.08557968 AU
Vr =  776.952339772136

Slice number :  8
No. of points = " 7183 " at 0.08623392 AU
Vr =  781.8356877810157

Slice number :  9
No. of points = " 7359 " at 0.08688816 AU
Vr =  786.6454674771

Slice number :  10
No. of points = " 7543 " at 0.0875424 AU
Vr =  791.3833400148033

Slice number :  11
No. of points = " 7715 " at 0.08819664 AU
Vr =  796.0485


Slice number :  94
No. of points = " 14343 " at 0.14249856 AU
Vr =  1034.8915131092328

Slice number :  95
No. of points = " 14373 " at 0.1431528 AU
Vr =  1036.6647278739804

Slice number :  96
No. of points = " 14397 " at 0.14380704 AU
Vr =  1038.421808407913

Slice number :  97
No. of points = " 14429 " at 0.14446128 AU
Vr =  1040.162973918402

Slice number :  98
No. of points = " 14479 " at 0.14511552 AU
Vr =  1041.8884396597143

Slice number :  99
No. of points = " 14525 " at 0.14576976 AU
Vr =  1036.5760440550569

Slice number :  100
No. of points = " 14551 " at 0.146424 AU
Vr =  1000.7822809566553

Slice number :  101
No. of points = " 14583 " at 0.14707824 AU
Vr =  968.4740552535974

Slice number :  102
No. of points = " 14603 " at 0.14773248 AU
Vr =  939.1481443742567

Slice number :  103
No. of points = " 14629 " at 0.14838672 AU
Vr =  912.3980350536895

Slice number :  104
No. of points = " 14659 " at 0.14904096 AU
Vr =  887.8799603642711

Slice number :  105
No. of points =


Slice number :  185
No. of points = " 15531 " at 0.2020344 AU
Vr =  445.96001641303576

Slice number :  186
No. of points = " 15519 " at 0.20268864 AU
Vr =  444.26906207211954

Slice number :  187
No. of points = " 15523 " at 0.20334288 AU
Vr =  442.5947216750369

Slice number :  188
No. of points = " 15515 " at 0.20399712 AU
Vr =  440.9339528923049

Slice number :  189
No. of points = " 15531 " at 0.20465136 AU
Vr =  439.29181976308075

Slice number :  190
No. of points = " 15543 " at 0.20530559999999998 AU
Vr =  437.66596661621645

Slice number :  191
No. of points = " 15541 " at 0.20595984 AU
Vr =  436.0552662022899

Slice number :  192
No. of points = " 15545 " at 0.20661407999999998 AU
Vr =  434.45955470101546

Slice number :  193
No. of points = " 15553 " at 0.20726832 AU
Vr =  432.878651126434

Slice number :  194
No. of points = " 15559 " at 0.20792255999999998 AU
Vr =  431.31241641433576

Slice number :  195
No. of points = " 15561 " at 0.2085768 AU
Vr =  429.75433600995933



In [80]:
images = []
num_frames = 240  # 
os.makedirs(output_dir, exist_ok=True)

for i in range(0,240):
    try:
        Br = np.loadtxt(f"{input_dir}FR_Br_{i}.txt")
        Bt = np.loadtxt(f"{input_dir}FR_Bt_{i}.txt")
        Bp = np.loadtxt(f"{input_dir}FR_Bp_{i}.txt")
        Vr = np.loadtxt(f"{input_dir}FR_Vr_{i}.txt")
        theta_pluto = np.loadtxt(f"{input_dir}FR_theta_{i}.txt")
        phi_pluto = np.loadtxt(f"{input_dir}FR_phi_{i}.txt")
        
        fig, axes = plt.subplots(2, 2, figsize=(10, 8), subplot_kw={'projection': 'polar'})
        fig.suptitle(f"Frame {i+1}")
        
        scatter_params = dict(cmap='viridis', s=10)
        
        valid_phi = (phi_pluto >= 0) & (phi_pluto <= 2 * np.pi)
        invalid_phi = ~valid_phi
        
        sc1 = axes[0, 0].scatter(phi_pluto[valid_phi], theta_pluto[valid_phi], c=Br[valid_phi], vmin=-60, vmax = 60, **scatter_params)
        axes[0, 0].scatter(phi_pluto[invalid_phi], theta_pluto[invalid_phi], color='r', s=10)
        axes[0, 0].set_title("Br")
        plt.colorbar(sc1, ax=axes[0, 0])
        
        sc2 = axes[0, 1].scatter(phi_pluto[valid_phi], theta_pluto[valid_phi], c=Bt[valid_phi], vmin=-60, vmax = 60, **scatter_params)
        axes[0, 1].scatter(phi_pluto[invalid_phi], theta_pluto[invalid_phi], color='r', s=10)
        axes[0, 1].set_title("Bt")
        plt.colorbar(sc2, ax=axes[0, 1])
        
        sc3 = axes[1, 0].scatter(phi_pluto[valid_phi], theta_pluto[valid_phi], c=Bp[valid_phi], vmin=-40, vmax = 40, **scatter_params)
        axes[1, 0].scatter(phi_pluto[invalid_phi], theta_pluto[invalid_phi], color='r', s=10)
        axes[1, 0].set_title("Bp")
        plt.colorbar(sc3, ax=axes[1, 0])
        
        sc4 = axes[1, 1].scatter(phi_pluto[valid_phi], theta_pluto[valid_phi], c=Vr[valid_phi], vmin=600, vmax = 1000, **scatter_params)
        axes[1, 1].scatter(phi_pluto[invalid_phi], theta_pluto[invalid_phi], color='r', s=10)
        axes[1, 1].set_title("Vr")
        plt.colorbar(sc4, ax=axes[1, 1])
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(f"{output_dir_gif}frame_input_{i+1}.png")
        images.append(f"{output_dir_gif}frame_input_{i+1}.png")
        plt.close()
        
        print(f"Saved frame_input_{i+1}.png")
    except Exception as e:
        print(f"Error processing frame {i}: {e}")

frames = [Image.open(f"{output_dir_gif}frame_input_{j}.png") for j in range(1,240)]

frames[0].save(outputgif, save_all=True, append_images=frames[1:],  duration=200, loop=0)
               
print( f"gif saved as {outputgif}")


Saved frame_input_1.png
Saved frame_input_2.png
Saved frame_input_3.png
Saved frame_input_4.png
Saved frame_input_5.png
Saved frame_input_6.png
Saved frame_input_7.png
Saved frame_input_8.png
Saved frame_input_9.png
Saved frame_input_10.png
Saved frame_input_11.png
Saved frame_input_12.png
Saved frame_input_13.png
Saved frame_input_14.png
Saved frame_input_15.png
Saved frame_input_16.png
Saved frame_input_17.png
Saved frame_input_18.png
Saved frame_input_19.png
Saved frame_input_20.png
Saved frame_input_21.png
Saved frame_input_22.png
Saved frame_input_23.png
Saved frame_input_24.png
Saved frame_input_25.png
Saved frame_input_26.png
Saved frame_input_27.png
Saved frame_input_28.png
Saved frame_input_29.png
Saved frame_input_30.png
Saved frame_input_31.png
Saved frame_input_32.png
Saved frame_input_33.png
Saved frame_input_34.png
Saved frame_input_35.png
Saved frame_input_36.png
Saved frame_input_37.png
Saved frame_input_38.png
Saved frame_input_39.png
Saved frame_input_40.png
Saved fra